In [1]:
%matplotlib inline
#The line above is necesary to show Matplotlib's plots inside a Jupyter Notebook
from PIL import Image
import numpy as np
from roipoly import RoiPoly
from EM import *
import cv2
# from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
import glob
import os
from skimage import io

print(os.getcwd())
print("hello world")



The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


/home/luminapursuit/miniconda3/envs/autoEnv/lib/python3.7/site-packages/pywt/_utils.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


/mnt/c/Users/Milan Gunasekera/dev/AutoSys/mag445_hmk1
hello world


In [10]:
%matplotlib inline
# %tb
from Barrel_ROI import *
from EM import *
import sys
import numpy as np
# import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
# from scipy import misc
import imageio
from scipy.stats import multivariate_normal, describe
import math
from scipy.linalg import det,inv
from scipy.stats import multivariate_normal
from skimage.measure import find_contours, regionprops # For barrel statistics
import cv2
import glob
import os

files = glob.glob('2019Proj1_train/*.png')
data_barrel = np.load('redbarrel_data.npy')
gauss = Model(data_barrel,3)

print(files[6])




bigmaskfiles = []



    


tmpdist = []
tmparea = []

def filler():
    
    for file in files:

        # predict distance
        distance = os.path.basename(file)[:-4] # This removes the filename (.png), just keeps the distance
    #     if '_' in distance: # This is for the image with two distances
    #         for d in distance.split('_'):
    #             tmp.append(float(d))
    #     else:
    #         tmp.append(float(distance))

        tmpdist.append(float(distance))


        # Predict where the barrel is
        img = io.imread(file)
        row,column, dim = img.shape

        samples = img.reshape((row*column,dim))
        tmp = np.array(gauss.likelihood(samples)).reshape((row,column)) # This is a 2D array, as it should be. This is magical, by the way.
        mask = np.ma.where(tmp > threshold, 255, 0)

        props = regionprops(mask) # Returns list, each one a certain region
        if props[0].area > 20000:
            bigmaskfiles.append((file,props[0].area))
        tmparea.append(props[0].area)
    dist = np.array(tmpdist)
    size = np.array(tmparea)



    # My bounding boxes suck
    minrow,mincol,maxrow,maxcol = props[0].bbox
    print(maxrow,mincol)
    print(props[0].centroid)


    # DO NOT ERASE!!!
    ax = plt.subplot(121)
    ax2 =plt.subplot(122)
    ax.imshow(img)
    # rect = patches.Rectangle((maxrow,mincol),maxcol-mincol,maxrow-minrow, linewidth=1,edgecolor='r',facecolor='none')
    rect = patches.Rectangle((maxrow,mincol),100,100, linewidth=1,edgecolor='r',facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)

    ax2.imshow(mask)

    plt.show()
    
    
    pass



def makemask(file):
    '''
    Creates the mask. Also creates the distance that it goes
    @param 
    Return:
    Image: actual
    mask
    '''
#     img = io.imread(file)
    threshold = -14 # This is arbitrarily set - It would probably be better if we could compare to other images
    tmp = cv2.imread(file)
    img = cv2.cvtColor(tmp,cv2.COLOR_BGR2RGB)
    data_barrel = np.load('redbarrel_data.npy')
    gauss = Model(data_barrel,3)
   
    
    row,column, dim = img.shape
    samples = img.reshape((row*column,dim))
    tmp = np.array(gauss.likelihood(samples)).reshape((row,column)) # This is a 2D array, as it should be. This is magical, by the way.
#     print(np.max(tmp,axis = 1), np.min(tmp,axis = 1))
    
    mask = np.uint8(np.where(tmp > threshold, 255, 0))
#     print(tmp.dtype)
#     retval, mask = cv2.threshold(tmp, 127, 255, cv2.THRESH_BINARY)
#     print(mask is None)
#     mask = tmp > threshold

    props = regionprops(mask) # Returns list, each one a certain region

    return img, mask

train_file = glob.glob('2019Proj1_train/*.png')
img,mask = makemask(train_file[0])
# ret, thresh = cv2.threshold(imgray, 50, 255, 0)
contours = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
print(contours[:5])
areas = [cv2.contourArea(c) for c in contours]
print(areas)




2019Proj1_train/2.6.png
(array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), [array([[[618, 406]],

       [[617, 407]],

       [[616, 407]],

       [[615, 407]],

       [[614, 407]],

       [[613, 408]],

       [[612, 408]],

       [[611, 407]],

       [[611, 408]],

       [[611, 409]],

       [[612, 410]],

       [[612, 411]],

       [[612, 412]],

       [[612, 413]],

       [[612, 414]],

       [[612, 415]],

       [[612, 416]],

       [[612, 417]],

       [[612, 418]],

       [[612, 419]],

       [[612, 420]],

       [[612, 421]],

       [[612, 422]],

       [[612, 423]],

       [[612, 424]],

       [[612, 425]],

       [[613, 426]],

       [[613, 427]],

       [[613, 428]],

       [[613, 429]],

       [[613, 430]],

       [[613, 431]],

       [[613, 432]],

       [[613, 433]],

       

error: OpenCV(3.4.2) /tmp/build/80754af9/opencv-suite_1535558553474/work/modules/imgproc/src/shapedescr.cpp:272: error: (-215:Assertion failed) npoints >= 0 && (depth == 5 || depth == 4) in function 'contourArea'


In [19]:
from scipy.linalg import pinv
from sklearn.linear_model import LinearRegression


def leastsquares_fit(dist,area):
    '''
    This function will approxmate the distance of something, given the area of it.
    
    inputs:
    dist: distance as np array. Convert to the inverse of the distance, this will be the model. This is dependent variable
    area: area, given as np array. This is independent variable
    
    return:
    Linear function, which is only two parameters
    '''
    y = 1 / dist
    X = size
    # Gives the prediction. Also puts the model into form that biases can be multiplied too
    A = np.vstack((size,np.ones(size.shape))).T
    
    beta = np.dot(pinv(A),y)
    
    # includes the bias term as well.
    return beta


def leastsquares_predict(model,values):
    '''
    Given a least square model, predict what the distance is. This is directly predicting the distance measure
    '''
#     sum_squared_residual = np.sum((y - leastsquares_predict(beta,X)) ** 2)
    
    A = np.vstack((values,np.ones(values.shape))).T
    return 1 / np.dot(A,model)
    

beta = leastsquares_fit(dist,size)
print(leastsquares_predict(beta,size))
print(dist)
# X = size.reshape(-1,1)
# y = dist
plt.figure(1)

plt.scatter(size,dist)
plt.show()
# print(dist)



NameError: name 'dist' is not defined

In [163]:
a = np.zeros((10,10))
a[2:4,2:4] = np.ones((2,2))
a[7:9,7:9] = np.ones((2,2))

# print(find_contours(a,0.5))
import os
file = files[0]
print(os.path.basename('2019Proj1_train/10.4.png'))
print(type(file))
print(os.path.basename(file))
print(file)
help(glob)




10.4.png
<class 'str'>
10.1.png
2019Proj1_train/10.1.png
Help on module glob:

NAME
    glob - Filename globbing utility.

MODULE REFERENCE
    https://docs.python.org/3.7/library/glob
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

FUNCTIONS
    escape(pathname)
        Escape all special characters.
    
    glob(pathname, *, recursive=False)
        Return a list of paths matching a pathname pattern.
        
        The pattern may contain simple shell-style wildcards a la
        fnmatch. However, unlike fnmatch, filenames starting with a
        dot are special cases that are not matched by '*' and '?'
        patterns.
        
        If recursive is true, the pattern '**' will match any files and
       

In [155]:
class Milan:
    def __init__(self,number,maximum):
        self.number = number
        self.maximum = maximum
    def __iter__(self):
        
        return a
    
    def __next__(self):
        
        '''
        This does something
        '''
        self.counter += 1
        value = self.number * self.counter

        if value &gt; self.maximum:
            raise StopIteration
        else:
            return value

if __name__ == '__main__':
    for number in Milan(463, 3000):
        print(number)

SyntaxError: invalid syntax (<ipython-input-155-c73749d9d097>, line 17)